In [607]:
import glob
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

#%matplotlib inline

# Load data of the different programming languages

In [608]:
program_languages = {
    'gcc': 'c', 
    'c': 'c',
    'csharp': 'csharp',
    'sbcl': 'common lisp',
    'clojure': 'clojure',
    'lhs': 'haskell',
    'java': 'java',
    'javascript': 'javascript',
    'ocaml': 'ocaml',
    'perl': 'perl',
    'hack': 'php', 
    'php': 'PHP',
    'python3': 'python',
    'py': 'python',
    'jruby': 'ruby', 
    'yarv': 'ruby',
    'scala': 'scala',
    'racket': 'scheme'
}

In [609]:
def file_search(file):
    files = glob.glob(file, recursive=True)
    text = []
    #print('This', files)
    for file in files:
        with open(file, encoding='latin_1') as f:
            text.append(f.read())

    return text

In [610]:
x_train = []
y_train = []
for key, vals in program_languages.items():
    x = file_search('benchmarksgame1/benchmarksgame/bench/**/*.{}'.format(key))
    x_train += x
    y_train += (len(x) * [vals.lower()])

## Teach or model the train data

In [611]:
pipeline = Pipeline([
        ('vect', CountVectorizer(analyzer='word', token_pattern=r'\w{2,}|\s{2,}|[^\w\d\s]')),
        ('clf', MultinomialNB()),
    ])

In [612]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\w{2,}|\\s{2,}|[^\\w\\d\\s]',
        tokenizer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [613]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.65, random_state=4)
pipeline.score(x_train, y_train)

0.9899749373433584

## Read in test data for predict stage

In [614]:
testdf = pd.read_csv('test.csv', names=['index', 'ynew_data'])

xtests = []
for number in testdf['index']:
    with open('test/{}'.format(number), encoding='latin_1') as f:
        xtests.append(f.read())

testdf['xtests'] = xtests
# len(ynew_data)
# len(testdf.ynew_data)

In [618]:
for _, series in testdf.iterrows():
    x = series['xtests']
    y = series['ynew_data']
    predicted_language = pipeline.predict([x])[0]
    #if predicted_language != y:
    #    print(predicted_language, y)

php javascript
scala javascript
c javascript
php ruby
scala haskell
clojure haskell
php haskell
c java
c java
php tcl
php tcl
c php


In [616]:
pipeline.score(xtests, testdf.ynew_data)

0.625

In [537]:
y_pred = pipeline.predict(testdf.xtests)
#y_pred.shape
#y_test.head()
#len(xtests)
#len(ynew_data)

In [619]:

accuracy_score(testdf.ynew_data, y_pred)

0.21875

## accuracy Score is quite low.  There are some issues bringing in the training data that I am still working on.  I believe once those issues are fixed scores should get up .7 or higher.